In [1]:
#---- Imports
using LinearAlgebra
using StaticArrays
using Pkg
using PyCall
using Plots
Plots.plotlyjs()
using Distances
using Statistics
theme(:dark)
p = pyimport("pymatgen")

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-6159691403499744796\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-13303769656450898037\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-6159691403499744796\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

PyObject <module 'pymatgen' from '/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/pymatgen/__init__.py'>

In [2]:
#= Defining structure for tetra
=#
struct tetra
    ge::Array{Float64,1}
    cl1::Array{Float64,1}
    cl2::Array{Float64,1}
    cl3::Array{Float64,1}
end
#check equavality of two struc 
function check_equal(t1::tetra,t2::tetra)
    if t1.ge==t2.ge && t1.cl1==t2.cl1  && t1.cl2==t2.cl2 && t1.cl3==t2.cl3
        return true
    else
        return false
        end;
    end;
#=------------------
Plotting Tools
-------------------=#
function plot_tetra(tetra::tetra,pl)
    lw=2
    pos1=[tetra.ge,tetra.cl1,tetra.cl2,tetra.cl3]
    scatter!(pl,[pos1[i][1] for i in [2,3,4]],
        [pos1[i][2] for i in [2,3,4]],
        [pos1[i][3] for i in [2,3,4]],color="darkblue",label="")
    scatter!(pl,[pos1[i][1] for i in [1]],
        [pos1[i][2] for i in [1]],
        [pos1[i][3] for i in [1]],color="darkred",label="")
    for i in 2:4
        plot!(pl,[pos1[1][1],pos1[i][1]],
            [pos1[1][2],pos1[i][2]],
            [pos1[1][3],pos1[i][3]],linewidth=lw,color="white",label="")
    plot!(pl,[pos1[i][1] for i in [2,3,4,2]],
            [pos1[i][2] for i in [2,3,4,2]],
            [pos1[i][3] for i in [2,3,4,2]],linewidth=lw,
            color="grey",linestyle=:dash,label="")
    end
end

plot_tetra (generic function with 1 method)

In [4]:
#= Mat mul for rotation
=#
icross(b) = copy(Transpose(hcat([cross(Matrix(1.0I, 3, 3)[:,i],b) for i in 1:3]...)));
anchor(tetra) = (tetra.ge+tetra.cl1+tetra.cl2+tetra.cl3)/4

function Mdot(a1,a2)
    #=redefing dot product like numpy for matrix=#
    a1_1=copy(Transpose(a1))'
    return [dot(a1_1[i,:],a2) for i in 1:3]
end


function rot(coords,anc,axis,theta)
    #=rotate coords by theta wrt anc with axis as axis=#
    theta %= 2 * pi
    rm=exp(icross(axis/norm(axis))*theta)
    val=Mdot(rm,(coords-anc))+anc
    return val
    end;


#ROtating a tetra
function rot_tetra_test_1(t::tetra,axis::Array{Float64,1},theta)
    #=rotate all atoms in tetra by theta with axis = axis=#
    anc=anchor(t)
    ge=rot(t.ge,anc,axis,theta)
    cl1=rot(t.cl1,anc,axis,theta)
    cl2=rot(t.cl2,anc,axis,theta)
    cl3=rot(t.cl3,anc,axis,theta)
    return tetra(ge,cl1,cl2,cl3)
    end;

#---PBC index
function pbc_1(system,index)
    eval_1(i,n)= ((abs(i)-1) ÷ n)*((sign(i)+1) ÷ 2) - ((sign(i-1)-1) ÷ 2)*((i-n) ÷ n)
    index_1(i,n)=((abs(i+n-1) % n)+1)
    unit_size=size(system)[1]
    a=(-system[1,1,1].ge+system[1,1,2].ge)[3]*unit_size
    uc=eval_1.(index,unit_size)*a
    index_mod=index_1.(index,unit_size)#index .% (unit_size+1)
    return index_mod,uc
    end;

#----get the tetra of a system at given index with PBC
function system_at_index(system,index)
    index_mod,uc=pbc_1(system,index)
    tetra_tmp=system[index_mod[1],index_mod[2],index_mod[3]]
    ge=tetra_tmp.ge+uc
    cl1=tetra_tmp.cl1+uc
    cl2=tetra_tmp.cl2+uc
    cl3=tetra_tmp.cl3+uc
    return tetra(ge,cl1,cl2,cl3)
    end;




# NN algorithm
function get_nn_2(system,pos)
    tmp=Array{Float64, 1}[]
    for i in -1:1
        for j in -1:1
            for k in -1:1
                index_tmp=pos+[i,j,k]
                temp=system_at_index(system,index_tmp)
                push!(tmp,temp.cl1)
                push!(tmp,temp.cl2)
                push!(tmp,temp.cl3)
            end
        end
    end
    return tmp
end

# Mean and Var from NN
function get_mean_var(system,pos,return_type="var")
    # Get the mean or variance of a position of lattice  wrt bond distance#
    sys=system[pos[1],pos[2],pos[3]]
    distance_1=colwise(Euclidean(), sys.ge, copy(hcat(get_nn_2(system,pos)...)))
    nn=sort(distance_1)[1:6]
    var_sys=var(nn)
    mean_sys=mean(nn)
    if return_type=="mean"
        return mean_sys
    else
        return var_sys
    end
end


# Make the system from CsSiI2.cif and pymatgen

function make_symstem(n=2)
    cssii2=p.Structure.from_file("Cssii2.cif")
    struc1=cssii2.copy()
    struc1.make_supercell([[n,0,0],[0,n,0],[0,0,n]])
    dist=cssii2.get_distance(1,3)+.1
    positions=Array{Int64, 1}[]
    for i in struc1
        if i.species_string=="Si"
            push!(positions,struc1.get_neighbor_list(dist,[i])[2])
        end
    end
    system=Array{tetra,3}(undef,n,n,n);
    cnt=1
    for i in 1:n
        for j in 1:n
            for k in 1:n
                ge=[round(i,digits=4) for i in get(struc1,reverse(positions[cnt])[1]).coords]
                cl1=[round(i,digits=4) for i in get(struc1,reverse(positions[cnt])[2]).coords]
                cl2=[round(i,digits=4) for i in get(struc1,reverse(positions[cnt])[3]).coords]
                cl3=[round(i,digits=4) for i in get(struc1,reverse(positions[cnt])[4]).coords]
                system[i,j,k]=tetra(ge,cl1,cl2,cl3)    
                cnt+=1
            end
        end
    end
    return system
end

# Function to get dipole orientation of a tetrahedron

function get_dipol_vec(tetra::tetra)
    return (tetra.ge - (tetra.cl1+tetra.cl2+tetra.cl3)/3)/norm(tetra.ge - (tetra.cl1+tetra.cl2+tetra.cl3)/3)
end

# Get the distance  vector between two tetrahedrons supplying pl will plot it in pl
function get_dipole_dist(tetra1::tetra,tetra2::tetra,pl=nothing)
    r1=(tetra1.ge +tetra1.cl1+tetra1.cl2+tetra1.cl3)/4
    r2=(tetra2.ge +tetra2.cl1+tetra2.cl2+tetra2.cl3)/4
    if pl == nothing
        return r1-r2
    else
        plot_tetra(tetra1,pl)
        plot_tetra(tetra2,pl)
        plot!(pl,[i[1] for i in [r1,r2]],[i[2] for i in [r1,r2]],[i[3] for i in [r1,r2]],color="red",linewidth=3,label="connecting vector r12")
        end;
    end;

#=--- dipole energy between two tetra hedrons
BigFloat is needed for accuracy. Been struggling for ages without that ! sigh. =#

function get_dipole_energy_between_tetra(tetra1::tetra,tetra2::tetra)
    r12=[BigFloat(i) for i in get_dipole_dist(tetra1,tetra2)]*1.0E-10
    d1=[BigFloat(i) for i in get_dipol_vec(tetra1)]*1.0E-10
    d2=[BigFloat(i) for i in get_dipol_vec(tetra2)]*1.0E-10
    four_pe0= 9 * 10^9 #1/4πe0 in N⋅m^2⋅C^−2
    return BigFloat(( dot(d1,d2) - 3*(dot(d1,r12)*dot(d2,r12))/norm(r12)^2 ) * four_pe0 * norm(r12)^-3)
    end;

#= Get dipole energy of a given system at pos
Need to check the other algorithm giving weired results=#
function get_dipole_energy(system,pos)
    #=
    To calculate NN distnce, but it is nn_dist=[6.067,8.58,10.508,12.134,13.566,14.861,17.16,18.201,21.017]
    n=3
    unique(sort(round.(vec([norm(get_dipole_dist(system[1,1,1],system[i,j,k])) for i in 1:1+n, j in 1:1+n, k in 1:1+n]),digits=3)))=#
    nn_dist=[6.067,8.58,10.508,12.134,13.566,14.861,17.16,18.201,21.017]
    NN=1 # NN to consider for ewald sum
    dipole_energy=[]
    tetra_origin=system_at_index(system,pos)
    nn=[[0,0,1],[0,1,0],[1,0,0],[0,0,-1],[0,-1,0],[-1,0,0]]
    for i in -2:2
        for j in -1:1
            for k in -1:1
                index_tmp=pos+[i,j,k]
                temp=system_at_index(system,index_tmp)
                if 0<3#norm(get_dipole_dist(temp,tetra_origin))<=nn_dist[NN]+.3
                    if ! check_equal(temp,tetra_origin)
                        tmp=get_dipole_energy_between_tetra(
                                        tetra_origin
                                            ,temp)
                        push!(dipole_energy,tmp)
                            end;
                    end;
                end;
            end;
        end;
#     for i in nn
#         index_tmp=pos+i
#         temp=system_at_index(system,index_tmp)
#         if ! check_equal(temp,tetra_origin)
#             tmp=get_dipole_energy_between_tetra(
#                             tetra_origin
#                                 ,temp)
#             push!(dipole_energy,tmp)
#             #print("$index_tmp $pos $tmp\n")
#             end;
#         end;
    return sum(dipole_energy)
end;


# Trial system

In [281]:
n=4;nk=200;
system=make_symstem(n);
mean_1=[]
var_1=[]
angle=LinRange(0, pi*2, nk)
for i in angle
    theta=i
    axis=[0,1,1.]
    system_test=copy(system)
    pos_1=[4,4,4]
    system_test[pos_1[1],pos_1[3],pos_1[3]]=
            rot_tetra_test_1(system[pos_1[1],pos_1[3],pos_1[3]],axis,theta);
    var_tmp=mean([get_mean_var(system_test,[i,j,k],"var") 
            for i in 1:n for j in 1:n for k in 1:n])
    mean_tmp=mean([get_mean_var(system_test,[i,j,k],"mean") 
            for i in 1:n for j in 1:n for k in 1:n])
    push!(mean_1,mean_tmp)
    push!(var_1,var_tmp)
    end;

In [282]:
pl_mean=plot(angle/(2π),mean_1,ylabel="mean",xlabel="angle (2π)",label="",linewidth=2)
pl_var=plot(angle/(2π),var_1,ylabel="var",xlabel="angle (2π)",label="",linewidth=2)
plot(pl_mean,pl_var)

<!DOCTYPE html>
 
 
 Plots.jl

In [5]:
function get_dipole_energy(system,pos)
    #=
    To calculate NN distnce, but it is nn_dist=[6.067,8.58,10.508,12.134,13.566,14.861,17.16,18.201,21.017]
    n=3
    unique(sort(round.(vec([norm(get_dipole_dist(system[1,1,1],system[i,j,k])) for i in 1:1+n, j in 1:1+n, k in 1:1+n]),digits=3)))=#
    nn_dist=[6.067,8.58,10.508,12.134,13.566,14.861,17.16,18.201,21.017]
    NN=1 # NN to consider for ewald sum
    dipole_energy=[]
    tetra_origin=system_at_index(system,pos)
    nn=[[0,0,1],[0,1,0],[1,0,0],[0,0,-1],[0,-1,0],[-1,0,0]]
    for i in -1:1
        for j in -1:1
            for k in -1:1
                index_tmp=pos+[i,j,k]
                temp=system_at_index(system,index_tmp)
                if norm(get_dipole_dist(temp,tetra_origin))<=nn_dist[NN]+.3
                    if ! check_equal(temp,tetra_origin)
                        tmp=get_dipole_energy_between_tetra(
                                        tetra_origin
                                            ,temp)
                        push!(dipole_energy,tmp)
                            end;
                    end;
                end;
            end;
        end;
#     for i in nn
#         index_tmp=pos+i
#         temp=system_at_index(system,index_tmp)
#         if ! check_equal(temp,tetra_origin)
#             tmp=get_dipole_energy_between_tetra(
#                             tetra_origin
#                                 ,temp)
#             push!(dipole_energy,tmp)
#             #print("$index_tmp $pos $tmp\n")
#             end;
#         end;
    return sum(dipole_energy)
end;

In [14]:
n=6;nk=300;
system=make_symstem(n);
dipole=[]
angle=LinRange(0, pi*4, nk)
for i in angle
    theta=i
    axis=[1,0,0.]
    system_test=copy(system)
    pos_1=[1,1,1]
    system_test[pos_1[1],pos_1[3],pos_1[3]]=
            rot_tetra_test_1(system[pos_1[1],pos_1[3],pos_1[3]],axis,theta);
    dipole_tmp=get_dipole_energy(system_test,pos_1);
    push!(dipole,dipole_tmp)
    end;

In [15]:
plot(angle/(2π),dipole,ylabel="Dipole (arb. units)",xlabel="angle (2π)",label="",linewidth=4)

<!DOCTYPE html>
 
 
 Plots.jl

In [ ]:
n=3;nk=300;
system=make_symstem(n);
function plot_perov(i)
    plt=plot(aspect_ratio=1,legend=false,size = (400, 400))
    theta=i
    axis=[0,1,1.]
    system_test=copy(system)
    pos_1=[2,2,2]
    system_test[pos_1[1],pos_1[3],pos_1[3]]=
            rot_tetra_test_1(system[pos_1[1],pos_1[3],pos_1[3]],axis,theta);
    for i1 in -1:1
        for j1 in -1:1
            for k1 in -1:1
                index_tmp=pos_1+[i1,j1,k1]
                temp=system_at_index(system_test,index_tmp)
                plot_tetra(temp,plt)
                end;end;end;
    return plt
end
nk=60
Plots.gr()
anim = @animate for i ∈ LinRange(0, pi*2, nk)
    plot_perov(i)
end
gif(anim, "rot_011.gif", fps = 15)